In [0]:
# provider-notebooks/Module 3 - Delta Sharing Architectural Patterns and Best Practices/Includes/Demo-Setup-3.1.ipynb

In [0]:
%run ./_common

In [0]:
# Set variables in python and sql
config_values = [
    ("catalog", "global_analytics"),
    ("schema", "transactions"),
    ("source_table", "transactions_master"),
    ("replica_table", "transactions_r2_replica"),
]
DA = create_da_objects(config_values)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS IDENTIFIER(DA.catalog)
COMMENT 'Provider catalog for cross-cloud replication demo';
USE CATALOG IDENTIFIER(DA.catalog);

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS IDENTIFIER(DA.schema)
COMMENT 'Schema containing transaction data for R2 replication';
USE SCHEMA IDENTIFIER(DA.schema);

In [0]:
display_config_values(config_values)